<center><h1>Testing Notebook</h1></center>


## Loading Entities:

 - Power System
 - Ant Colony

In [ ]:
from acopoweropt import system, colony

PSystem = system.PowerSystem(name='s10')
Colony = colony.Colony(n_ants=5,phr_evp_rate=0.25)

## Testing Power System

In [ ]:
# Randomly select an operation
operation = PSystem.sample_operation()

# Solve system
solution = PSystem.solve(operation=operation)

## Testing Colony

In [1]:
# main.py
import pandas as pd
import random

from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s15')
Colony = colony.PowerColony(n_ants=5,
                            pheromone_evp_rate={'worst': 0.75, 'mean': 0.25, 'best': 0.05},
                            power_system=PSystem)

n_iter = 10

for i in range(n_iter):
    # Getting last taken paths
    paths = Colony.paths.query("iteration == {i}".format(i=i))

    # Evaporate Pheromone
    Colony.evaporate_pheromone(paths=paths, power_system=PSystem)
    
    print(i)
    print(Colony.pheromone)

    # Selecting Ants (80/20) to follow or not the pheromone paths
    taken_paths = []
    for ant in range(1, Colony.n_ants + 1):
        decision = random.random()
        if decision <= 0.8:
            # ANT Follows Path
            operative_zones = Colony.choose_path()
            existing_path = paths.query("path == '{}'".format(",".join([str(n) for n in operative_zones])))
            
            # Check if chosen path was already calculated:
            if existing_path.values.size != 0:
                solution = existing_path.reset_index().loc[0]
                solution['ant'] = ant
                solution['iteration'] = i+1
                solution = solution.to_frame().T.set_index('ant')
                taken_paths.append(solution)
            else:
                operation = PSystem.get_operation(operative_zones=operative_zones)
                solution = pd.DataFrame(
                    [colony.seek_food(ant=ant, iteration=i+1, operation=operation, power_system=PSystem)]
                ).set_index('ant')
                taken_paths.append(solution)

        else:
            # ANT Sets to new path
            operation = PSystem.sample_operation()
            solution = pd.DataFrame(
                [colony.seek_food(ant=ant, iteration=i+1, operation=operation, power_system=PSystem)]
            ).set_index('ant')
            taken_paths.append(solution)

    # Updating Paths
    new_paths = pd.concat(taken_paths).sort_index()
    Colony.paths = Colony.paths.append(new_paths)
    
    # Updating Best and Worst Paths
    Colony.update_best_and_worst(paths=new_paths)

    # Updating Pheromone
    Colony.update_pheromone(paths=new_paths, iteration=i+1)

0
           1         2         3         4         5         6         7   \
opz                                                                         
1    0.341712  0.051495  0.341712  0.341712  0.088256  0.048055  0.341712   
2    0.000000  0.095917  0.000000  0.000000  0.028597  0.084394  0.000000   
3    0.000000  0.007357  0.000000  0.000000  0.015191  0.036096  0.000000   
4    0.000000  0.070776  0.000000  0.000000  0.109797  0.029053  0.000000   

           8         9         10        11        12        13        14  \
opz                                                                         
1    0.341712  0.341712  0.341712  0.341712  0.048836  0.341712  0.341712   
2    0.000000  0.000000  0.000000  0.000000  0.029711  0.000000  0.000000   
3    0.000000  0.000000  0.000000  0.000000  0.029327  0.000000  0.000000   
4    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

           15  
opz            
1    0.341712  
2    0.000000  
3    0.0

In [3]:
Colony.best_and_worst

,iteration,path,status,distance
ant,,,,
best,97,"1,4,1,1,3,4,1,1,1,1,1,2,1,1,1",optimal,32544.969575
worst,93,"1,1,1,1,1,1,1,1,1,1,1,1,1,1,1",optimal,33326.171532


In [5]:
Colony.pheromone_history

{0:            1         2         3         4         5         6         7   \
 opz                                                                         
 1    0.953209  0.216998  0.953209  0.953209  0.278932  0.113908  0.953209   
 2    0.000000  0.227360  0.000000  0.000000  0.160677  0.266728  0.000000   
 3    0.000000  0.055117  0.000000  0.000000  0.151742  0.202810  0.000000   
 4    0.000000  0.298248  0.000000  0.000000  0.195194  0.217650  0.000000   
 
            8         9         10        11        12        13        14  \
 opz                                                                         
 1    0.953209  0.953209  0.953209  0.953209  0.274393  0.953209  0.953209   
 2    0.000000  0.000000  0.000000  0.000000  0.222581  0.000000  0.000000   
 3    0.000000  0.000000  0.000000  0.000000  0.219705  0.000000  0.000000   
 4    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 
            15  
 opz            
 1    0.953209  
 2    

In [4]:
optimal = [1,3,1,1,3,4,1,1,1,1,1,2,1,1,1]
test = [1,2,1,1,2,3,1,1,1,1,1,2,1,1,1]

operation = PSystem.get_operation(operative_zones=test)

solution = PSystem.solve(operation=operation)

pd.DataFrame([Colony.seek_food(ant=1, operation=operation, PowerSystem=PSystem)]).set_index('ant')

,path,status,distance,tau
ant,,,,
1,"1,2,1,1,2,3,1,1,1,1,1,2,1,1,1",optimal,32702.10893,0.000031
